In [7]:
import torchmetrics
import torchmetrics.functional as mF
import torch
from monai.losses import DiceLoss

dsc_loss_w = 1.0

Dice = torchmetrics.Dice()
DiceFG = torchmetrics.Dice(ignore_index=0) #ignore_index=0 means we ignore the background class
# MonaiDiceLoss = 
# nnUnetDiceLoss = 


In [12]:
# Creating Dummy Test Tensors for testing of same format as in the model [B, C, H, W, D]
# B = Batch Size, C = Number of Classes, H = Height, W = Width, D = Depth
# Example with Batch Size 1, 4 Classes, Height 3, Width 3, Depth 3
n_classes = 4
preds = torch.tensor([[0, 1, 1], [1, 2, 0], [1, 2, 0]])
masks = torch.tensor([[1, 2, 3], [0, 1, 1], [0, 0, 1]])

In [8]:
# Testing with example tensors from terminal
# unique values in masks: [0 2 3]
# unique values in preds: [0 2]
n_classes = 4
preds = torch.tensor([[0, 0, 2], [2, 2, 3], [0, 2, 0]])
masks = torch.tensor([[0, 0, 2], [2, 2, 0], [0, 2, 2]])

In [9]:
dice_p_cls = mF.dice(preds, masks, average=None, num_classes=n_classes) # average=None returns dice per class
print(dice_p_cls)

# -> dice for class 1 should be 1 as predictions didn't contain the class which was also absent in the ground truth

tensor([0.7500,    nan, 0.8889, 0.0000])


In [13]:
# Dice Scores:
dice = mF.dice(preds, masks)
diceFG = mF.dice(preds, masks, ignore_index=0)
dice_p_cls = mF.dice(preds, masks, average=None, num_classes=n_classes) # average=None returns dice per class

# ET (Enhancing Tumor): label 3
dice_ET = mF.dice((preds == 3), (masks == 3), ignore_index=0)

# TC(Tumor Core): ET + NCR = label 1 + label 3
dice_TC = mF.dice((preds == 1) | (preds == 3), (masks == 1) | (masks == 3), ignore_index=0)

# WT (Whole Tumor): TC + ED = label 1 + label 2 + label 3
dice_WT = mF.dice((preds > 0), (masks > 0), ignore_index=0)

print("Dice Score: ", dice)
print("Dice Score FG: ", diceFG)
print("Dice Score per Class: ", dice_p_cls)
print("Dice Score ET: ", dice_ET)
print("Dice Score TC: ", dice_TC)
print("Dice Score WT: ", dice_WT)

Dice Score:  tensor(0.)
Dice Score FG:  tensor(0.)
Dice Score per Class:  tensor([0., 0., 0., 0.])
Dice Score ET:  tensor(0.)
Dice Score TC:  tensor(0.2222)
Dice Score WT:  tensor(0.5000)


In [7]:
#Dice Losses
# Brats Dice Loss (Sum of dice_ET, dice_TC, dice_WT divided by 3)
dice_ET_loss = (1 - mF.dice((preds == 3), (masks == 3))) * dsc_loss_w / 3
dice_TC_loss = (1 - mF.dice((preds == 1) | (preds == 3), (masks == 1) | (masks == 3)) ) * dsc_loss_w / 3
dice_WT_loss = (1 - mF.dice((preds > 0), (masks > 0))) * dsc_loss_w / 3

dice_loss = (1-mF.dice(preds, masks)) * dsc_loss_w



print("Dice ET Loss: ", dice_ET_loss)
print("Dice TC Loss: ", dice_TC_loss)
print("Dice WT Loss: ", dice_WT_loss)
print("Dice Loss: ", dice_loss)

Dice ET Loss:  tensor(0.)
Dice TC Loss:  tensor(0.1481)
Dice WT Loss:  tensor(0.0741)
Dice Loss:  tensor(0.4444)
